In [64]:
import pandas as pd

df_test = pd.read_csv("test.csv") # does not have asnwer metrics (`scores` in this case)
df_train = pd.read_csv("train.csv")
df_train.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,1057,1,2.7,0,3,1,41,0.1,97,4,...,397,891,2033,16,9,2,1,Yes,0,1
1,1619,1,1.9,0,2,1,21,0.4,138,1,...,46,562,1641,7,2,13,1,No,0,1
2,1028,1,2.8,0,1,1,30,0.1,193,1,...,1801,1923,1424,18,4,13,1,No,1,1
3,1994,1,0.8,1,2,1,7,0.6,88,8,...,667,711,454,11,8,6,1,Yes,0,0
4,1603,1,0.5,1,1,1,17,0.5,114,2,...,430,1207,1637,17,3,11,1,No,1,1


В скольких столбцах таблицы среднее значение измеряется в сотнях? (не в десятках и не в тысячах).

In [65]:
ncat = df_train.drop('touch_screen', axis=1)
ncat.loc[:, (ncat.mean() > 99) & (ncat.mean() < 1000)]

,mobile_wt,px_height
0,97,397
1,138,46
2,193,1801
3,88,667
4,114,430
...,...,...
1495,100,1134
1496,139,570
1497,119,545
1498,80,815


Задание 2 (0.3 балла). Если у телефона нет ни опции 3G, ни опции 4G, ни Wifi - то у пользователей нет доступа к интернету. Создайте колонку is_internet и поставьте туда 0, если у телефона нет ни одной из трех перечисленных опций, и 1 иначе.

Такую же колонку создайте в тестовых данных.

Какая доля телефонов (из train.csv) не имеет доступа к интернету? Ответ округлите до сотых.

In [66]:
df_train['is_internet'] = ((df_train['four_g'] + df_train['three_g'] + df_train['wifi']) > 0)
df_train['is_internet'] = df_train['is_internet'].astype(int)

df_test['is_internet'] = ((df_test['four_g'] + df_test['three_g'] + df_test['wifi']) > 0)
df_test['is_internet'] = df_test['is_internet'].astype(int)

round(len(df_train[df_train['is_internet'] == 0]) / len(df_train), 2)

0.12

Задание 3 (каждый пункт - 0.42 балла, 1.25 балла максимум)
a. Какой столбец имеет наибольшую по модулю корреляцию Пирсона с целевой переменной price_range? В ответ запишите коэффициент корреляции (со знаком), округленный до сотых.

In [67]:
ncat.corrwith(ncat['price_range'], method='pearson')

battery_power    0.200020
blue             0.023878
clock_speed      0.012216
dual_sim         0.020301
fc              -0.000835
four_g           0.016724
int_memory       0.050547
m_dep           -0.000310
mobile_wt       -0.031402
n_cores         -0.005716
pc               0.001921
px_height        0.154673
px_width         0.172939
ram              0.916218
sc_h             0.039723
sc_w             0.053221
talk_time        0.002455
three_g          0.024530
wifi             0.027465
price_range      1.000000
dtype: float64

b. Среди телефонов ценовой категории 2 (price_range = 2) вычислите долю тех, которые не имеют bluetooth. Ответ округлите до сотых.

In [68]:
pr2 = df_train[df_train['price_range'] == 2]
pr2blue = pr2[pr2['blue'] == 0]

round(len(pr2blue) / len(pr2), 2)

0.52

c. Среди телефонов с talk_time < 10 насколько больше телефонов с опцией dual_sim = 1, чем с dual_sim = 0?

In [69]:
tt10 = df_train[df_train['talk_time'] < 10]
dd1 = tt10[tt10['dual_sim'] == 1]
dd0 = tt10[tt10['dual_sim'] == 0]

len(dd1) - len(dd0)

48

a. (0.3 балла)

Разбейте тренировочные данные на целевой вектор y, содержащий значения из столбца price_range, и матрицу объект-признак X, содержащую остальные признаки.

Посмотрите, сколько значений содержит категориальный столбец touch_screen. Значения, встречающиеся меньше, чем в 1% строк, замените на самое частое значение. После этого закодируйте столбец при помощи OneHot-encoding (используйте аргумент drop='first'). Обучайте OHE на тренировочных данных X, но кодируйте при этом и тренировочные, и тестовые данные.  Для кросс-валидации лучше использовать функцию cross_val_score из библиотеки sklearn.model_selection.

Сколько изначально различных значений было в столбце touch_screen?

In [70]:
# 1. Проверим уникальные значения
unique_values = df_train['touch_screen'].unique()
print("Уникальные значения:", unique_values)

# 2. Определим частоту значений
value_counts = df_train['touch_screen'].value_counts()
total_rows = len(df_train)
threshold = 0.01 * total_rows

# Найдём самое частое значение
most_frequent_value = value_counts.idxmax()

# Заменим значения, встречающиеся реже, чем в 1% строк
values_to_replace = value_counts[value_counts < threshold].index
df_train['touch_screen'] = df_train['touch_screen'].replace(values_to_replace, most_frequent_value)

# 3. Закодируем столбец touch_screen при помощи OneHot-encoding
df_train_encoded = pd.get_dummies(df_train, columns=['touch_screen'], drop_first=True)

# Выделим матрицу объект-признак X и вектор ответов y
X = df_train_encoded.drop('price_range', axis=1)
y = df_train_encoded['price_range']

X

Уникальные значения: ['Yes' 'No' '-']


,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,wifi,is_internet,touch_screen_Yes
0,1057,1,2.7,0,3,1,41,0.1,97,4,...,397,891,2033,16,9,2,1,0,1,True
1,1619,1,1.9,0,2,1,21,0.4,138,1,...,46,562,1641,7,2,13,1,0,1,False
2,1028,1,2.8,0,1,1,30,0.1,193,1,...,1801,1923,1424,18,4,13,1,1,1,False
3,1994,1,0.8,1,2,1,7,0.6,88,8,...,667,711,454,11,8,6,1,0,1,True
4,1603,1,0.5,1,1,1,17,0.5,114,2,...,430,1207,1637,17,3,11,1,1,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,1742,1,1.6,0,3,0,49,0.8,100,1,...,1134,1469,1947,5,4,6,1,1,1,False
1496,1936,0,1.3,0,2,0,44,0.1,139,3,...,570,1021,1400,17,7,3,0,1,1,False
1497,1567,0,0.5,1,3,0,22,0.5,119,6,...,545,1621,2312,9,1,18,1,0,1,False
1498,1456,0,1.6,1,9,1,39,0.1,80,8,...,815,1666,3998,6,3,13,1,0,1,True


b (0.4 балла) Обучите на этих данных логистическую регрессию из sklearn (LogisticRegression) с параметрами по умолчанию. Выведите среднее значение метрики f1_score c вариантом усреднения ‘weighted’  (или же 'f1_weighted') алгоритма на кросс-валидации с тремя фолдами. Ответ округлите до сотых.

При объявлении модели задайте random_state = 42.
penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None

In [71]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

lr = LogisticRegression(random_state = 42, penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
round(cross_val_score(lr, X, y, cv=3, scoring='f1_weighted').mean(), 2)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

0.63

a. Подберите значение константы регуляризации C в логистической регрессии, перебирая гиперпараметр от 0.001 до 100 включительно, проходя по степеням 10. Для выбора С примените перебор по сетке по тренировочной выборке (GridSearchCV из библиотеки sklearn.model_selection) с тремя фолдами и метрикой качества - f1_weighted. Остальные параметры оставьте по умолчанию. В ответ запишите наилучшее среди искомых значение С. (0.25 балла)

При объявлении модели задайте random_state = 42.

Комментарий: параметры по умолчанию предполагаются следующими:

penalty='l2', dual=False, tol=0.0001, fit_intercept=True, intercept_scaling=1, class_weight=None,  solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None

In [73]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
grid_search = GridSearchCV(estimator=lr, param_grid=param_grid, cv=3, scoring='f1_weighted')

grid_search.fit(X, y)
C = grid_search.best_params_['C']
print(f"$$$$$$$$$$$$$$$$$: {C}")

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

$$$$$$$$$$$$$$$$$: 1


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
